In [1]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import os

In [2]:
idx = [(1, 0), (4, 1), (5, 2), (6, 3), (7, 4), (12, 5), (13, 6), (14, 7), (19, 8)]
for z, c in idx:
    data_set = pickle.load(open(r'label' + str(z) + '\goal_set.p', 'rb'))
    data = data_set['train'] + data_set['test'] + data_set['validate']
    dis_s = {}
    symptoms = {}
    for xy in data:
        disease = xy['disease_tag']
        dis_s.setdefault(disease, {'index':len(dis_s), 'symptom':dict()})
        temp2 = xy['goal']['implicit_inform_slots']
        temp2.update(xy['goal']['explicit_inform_slots'])
        for x, y in temp2.items():
            symptoms.setdefault(x, len(symptoms))
            dis_s[disease]['symptom'].setdefault(x, 0)
            dis_s[disease]['symptom'][x] += 1
    #for x in dis_s:
        #frequency = 0
        #for y in dis_s[x]['symptom']:
            #frequency += dis_s[x]['symptom'][y]
        #for y in dis_s[x]['symptom']:
            #dis_s[x]['symptom'][y] /= frequency
    symptoms['Total Symptom'] = len(symptoms)
    pickle.dump(obj = dis_s, file = open(r'label' + str(z) + '\out1.p', 'wb'), protocol = 2)
    pickle.dump(obj = symptoms, file = open(r'label' + str(z) + '\out2.p', 'wb'), protocol = 2)

In [3]:
idx = [(1, 0), (4, 1), (5, 2), (6, 3), (7, 4), (12, 5), (13, 6), (14, 7), (19, 8)]
#idx = [(1, 0)]
for z, c in idx:
    print(z)
    temp_dict = pickle.load(open(r'label' + str(z) + '\out2.p', 'rb'))
    temp_data = pickle.load(open(r'label' + str(z) + '\out1.p', 'rb'))
    length = 0

    dict = {}
    dict1 = {}
    dict2 = {}
    dict3 = {}

    for xy in temp_dict:
        dict[length] = xy
        dict1[xy] = length
        length +=1

    length = 0
    for xy in temp_data:
        dict2[length] = xy
        dict3[xy] = length
        length += 1
        
    data_set = pickle.load(open(r'label' + str(z) + '\goal_set.p', 'rb'))
    data = data_set['train'] + data_set['test'] + data_set['validate']
    count1 = 0
    
    for xy in data:
        count1 += 1
    X = np.zeros(shape=(count1, temp_dict["Total Symptom"]), dtype=np.uint8)
    y = np.zeros(shape=(1, count1), dtype=np.uint8)
    y = y.flatten()
    count1 = 0
    
    
    for xy in data:
        for xx in xy['goal']['explicit_inform_slots']:
            X[count1][dict1[xx]] = 1
        for xx in xy['goal']['implicit_inform_slots']:
            X[count1][dict1[xx]] = 1
        y[count1] = dict3[xy['disease_tag']]
        count1 += 1
    
    #model = ExtraTreesClassifier(n_estimators = 10)
    #model.fit(X_train, y_train)
    #a = model.feature_importances_
    tot = int(temp_dict["Total Symptom"])
    tot = int((80*tot + 99)/100)
    bestfeatures = SelectKBest(score_func=chi2, k = tot)
    print(tot)
    fit = bestfeatures.fit(X,y)
    X = bestfeatures.transform(X)
    dfscores = fit.scores_
    #print(dfscores)
    l = []
    timer = 0
    for i in dfscores:
        l.append((i, dict[timer]))
        timer += 1
    l.sort()
    res = {}
    for i, j in l:
        res[j] = float(i)
    pickle.dump(obj = res, file = open(r'label' + str(z) + '\out3.p', 'wb'), protocol = 2)
    l.reverse()
    r = []
    res = {}
    for i in range (0, tot):
        r.append(l[i])
    res = {}
    timer = 0
    for i, j in r:
        res[j] = timer
        timer += 1
    pickle.dump(obj = res, file = open(r'label' + str(z) + '\out4.p', 'wb'), protocol = 2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    classifier = LogisticRegression(solver='liblinear', random_state = 0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(accuracy_score(y_test, y_pred))

1
52
0.862
4
72
0.984
5
55
0.847
6
47
0.934
7
37
0.782
12
41
0.796
13
50
0.938
14
56
0.955
19
59
0.87


In [4]:
idx = [(1, 0), (4, 1), (5, 2), (6, 3), (7, 4), (12, 5), (13, 6), (14, 7), (19, 8)]
x = []
for z, c in idx:
    temp_dict = pickle.load(open(r'label' + str(z) + '\out2.p', 'rb'))
    temp_data = pickle.load(open(r'label' + str(z) + '\out1.p', 'rb'))
    length = 0

    dict0 = {}
    dict1 = {}
    dict2 = {}
    dict3 = {}

    for xy in temp_dict:
        dict0[length] = xy
        dict1[xy] = length
        length += 1
    length = 0
    for xy in temp_data:
        dict2[length] = xy
        dict3[xy] = length
        length += 1
        
    #data_set = pickle.load(open(r'label' + str(z) + '\goal_set.p', 'rb'))
    #data = data_set['train'] + data_set['test'] + data_set['validate']
    X = np.zeros(shape=(length, temp_dict["Total Symptom"]), dtype=np.uint8)
    
    for xy in temp_data:
        for xx in temp_data[xy]['symptom']:
            X[dict3[xy]][dict1[xx]] = 1
    
    l = {}
    len = 0
    
    for xx in range(0, temp_dict["Total Symptom"]):
        cur = 0
        r = {}
        for yy in range(0, length):
            if X[yy][xx] == 1:
                r[dict2[yy]] = cur
                cur += 1
        l[dict0[xx]] = r
    
    pickle.dump(obj = l, file = open(r'label' + str(z) + '\out5.p', 'wb'), protocol = 2)